#### ID 2140

```American Express is reviewing their customers' transactions, and you have been tasked with locating the customer who has the third highest total transaction amount. The output should include the customer's id, as well as their first name and last name. For ranking the customers, use type of ranking with no gaps between subsequent ranks.```

In [ ]:
%%sql
WITH cte AS (SELECT cust_id,
                    SUM(total_order_cost)                              AS total_cost,
                    DENSE_RANK() OVER (ORDER BY SUM(total_order_cost)) AS rnk
             FROM card_orders
             GROUP BY cust_id)
SELECT cust_id, first_name, last_name
FROM cte
         JOIN customers AS c ON cte.cust_id = c.id
WHERE rnk = 3

In [ ]:
df = card_orders

groped_df = df.groupby('cust_id', as_index=False).agg(total_cost=('total_order_cost', 'sum'))

groped_df['rnk'] = groped_df['total_cost'].rank(method='dense')

groped_df.query('rnk == 3').merge(customers, how='inner', left_on='cust_id', right_on='id')[
    ['id', 'first_name', 'last_name']]

#### ID 2141

```Amazon's information technology department is looking for information on employees' most recent logins. The output should include all information related to each employee's most recent login.```

In [ ]:
%%sql
WITH cte
         AS (SELECT DENSE_RANK()
                    OVER (PARTITION BY worker_id ORDER BY login_timestamp DESC),
                    id,
                    worker_id,
                    login_timestamp,
                    ip_address,
                    country,
                    region,
                    city,
                    device_type
             FROM worker_logins)
SELECT id,
       worker_id,
       login_timestamp,
       ip_address,
       country,
       region,
       city,
       device_type
FROM cte
WHERE dense_rank = 1
ORDER BY id

In [ ]:
df = worker_logins

df['rnk'] = df.groupby('worker_id')['login_timestamp'].rank(method='first', ascending=False)

df.query('rnk == 1').drop(columns=['rnk'])

#### ID 2142

```You've been asked by Amazon to find the shipment_id and weight of the third heaviest shipment. Output the shipment_id, and total_weight for that shipment_id. In the event of a tie, do not skip ranks.```

In [ ]:
%%sql
WITH cte AS (SELECT shipment_id,
                    SUM(weight) AS total_weight,
                    DENSE_RANK() OVER (ORDER BY SUM(weight) DESC)
             FROM amazon_shipment
             GROUP BY shipment_id)
SELECT shipment_id, total_weight
FROM cte
WHERE dense_rank = 3

In [ ]:
df = amazon_shipment

grouped_df = df.groupby('shipment_id', as_index=False).agg(total_weight=('weight', 'sum'))

grouped_df['rnk'] = grouped_df['total_weight'].rank(method='dense', ascending=False)

grouped_df.query('rnk == 3').drop(columns=['rnk'])

#### ID 2143

```Bank of Ireland has requested that you detect invalid transactions in December 2022. An invalid transaction is one that occurs outside of the bank's normal business hours. The following are the hours of operation for all branches: Monday - Friday 09:00 - 16:00 Saturday & Sunday Closed Irish Public Holidays 25th and 26th December Determine the transaction ids of all invalid transactions.```

In [ ]:
%%sql
SELECT transaction_id
FROM boi_transactions
WHERE transaction_id IN (SELECT transaction_id
                             FROM boi_transactions
                             WHERE EXTRACT(MONTH FROM time_stamp) = 12
                               AND EXTRACT(DAY FROM time_stamp) IN (25, 26)
                             UNION
                             SELECT transaction_id
                             FROM boi_transactions
                             WHERE EXTRACT(DOW FROM time_stamp) IN (6, 0)
                             UNION
                             SELECT transaction_id
                             FROM boi_transactions
                             WHERE EXTRACT(DOW FROM time_stamp) BETWEEN 1 AND 5
                               AND EXTRACT(HOUR FROM time_stamp) NOT BETWEEN 9 AND 15)

In [ ]:
df = boi_transactions

holidays = df.query('time_stamp.dt.month == 12 & time_stamp.dt.day in ([25, 26])')['transaction_id'].to_list()

weekends = df.query('time_stamp.dt.dayofweek in ([5, 6])')['transaction_id'].to_list()

non_work_hours = df.query('time_stamp.dt.dayofweek.between(0, 4) & ~time_stamp.dt.hour.between(9, 15)')[
    'transaction_id'].to_list()

combined_list = holidays + weekends + non_work_hours

df.query('transaction_id.isin(@combined_list)')['transaction_id']

#### ID 2144

```A major airline has enlisted Tata Consultancy's help to improve customer satisfaction on its flights. Their goal is to increase customer satisfaction among people between the ages of 30 and 40. You've been tasked with calculating the customer satisfaction average for this age group across all three flight classes for 2022. Return the class with the average of satisfaction rounded to the nearest whole number. Note: Only survey results from flights in 2022 are included in the dataset.```

In [ ]:
%%sql
SELECT class, ROUND(AVG(satisfaction)) AS pc_score
FROM survey_results AS sr
         JOIN loyalty_customers AS lc ON sr.cust_id = lc.cust_id
WHERE age BETWEEN 30 AND 39
GROUP BY class

In [ ]:
df = pd.merge(survey_results, loyalty_customers, how='inner', on='cust_id')

df.query('age.between(30, 39)').groupby('class', as_index=False).agg(avg_score=('satisfaction', 'mean')).round()

#### ID 2151

```You have been asked to find the number of employees hired between the months of January and July in the year 2022 inclusive. Your output should contain the number of employees hired in this given time frame.```

In [ ]:
%%sql
SELECT COUNT(id) AS hired_emp
FROM employees
WHERE DATE_TRUNC('month', joining_date) BETWEEN '2022-01-01' AND '2022-07-01'

In [ ]:
df = employees

df[df['joining_date'].dt.to_period('M').between('2022-01-01', '2022-07-01')]['id'].count()

#### ID 2156

```You have been tasked with finding the worker IDs of individuals who logged in between the 13th to the 19th inclusive of December 2021. In your output, provide the unique worker IDs for the dates requested.'```

In [ ]:
%%sql
SELECT DISTINCT worker_id
FROM worker_logins
WHERE login_timestamp BETWEEN '2021-12-13' AND '2021-12-20'

In [ ]:
df = worker_logins

df.query('login_timestamp.between("2021-12-13", "2021-12-19")')['worker_id'].drop_duplicates()

#### ID 2159

```You have been asked to get a list of all the sign up IDs with transaction start dates in either April or May. Since a sign up ID can be used for multiple transactions only output the unique ID. Your output should contain a list of non duplicated sign-up IDs.```

In [ ]:
%%sql
SELECT DISTINCT signup_id
FROM transactions
WHERE DATE_TRUNC('month', transaction_start_date)::DATE IN ('2020-04-01', '2020-05-01')

In [ ]:
df = transactions

df.query('transaction_start_date >= "2020-04-01" & transaction_start_date < "2020-06-01"')['signup_id'].unique()

#### ID 2160

```The sales division is investigating their sales for the past month in Oregon. Calculate the total revenue generated from Oregon-based customers for April.```

In [ ]:
%%sql
SELECT SUM(cost_in_dollars * units_sold) AS total_sales
FROM online_orders AS oo
         JOIN online_customers AS oc ON oo.customer_id = oc.id
WHERE state ILIKE 'oregon'
  AND DATE_TRUNC('month', date) = '2022-04-01'

In [ ]:
df = pd.merge(online_orders, online_customers, how='inner', left_on='customer_id', right_on='id')

df['sales'] = df['cost_in_dollars'] * df['units_sold']

df.query('state == "Oregon" & date.between("2022-04-01", "2022-04-30")')['sales'].sum()